In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [3]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block

In [4]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    #solver = HistGradientBoostingRegressor() #TRYING DIFFERNT SOLVER TO ENSURE CONVERGENCE



    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'    
    default_output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': None,
        'tau': None,
        'theta': None,
        'E_tau_theta_results': None,
        'ccm_value': None,
        'convergence_p_value': None,
        'correlation': None
    }
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        try:
            c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        except:
            print(f"{interaction} did not converge")
            return default_output
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    try:
        convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)
    except:
        print(f"{interaction} did not converge")
        return default_output

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]# Changed end point from 1 to 0
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [24]:
HAB_data = pd.read_csv('Data/orig_block.csv', index_col=0) #LOAD DATA
HAB_data.columns

Index(['chlA_.1wk', 'chlA', 'nitrate', 'phosphate', 'silicate', 'nitrite',
       'nitrate_phosphate_ratio', 'N_star', 'Si_star', 'AvgTemp_1wk',
       'AvgDens_1wk', 'SST', 'WIND_SPD', 'U_WIND', 'PRCP_2wkavg',
       'AWND_2wkavg', 'chlA_1wk', 'nitrate_1wk', 'phosphate_1wk',
       'silicate_1wk', 'nitrite_1wk', 'nitrate_phosphate_ratio_1wk',
       'N_star_1wk', 'Si_star_1wk', 'AvgTemp_1wk_1wk', 'AvgDens_1wk_1wk',
       'SST_1wk', 'WIND_SPD_1wk', 'U_WIND_1wk', 'PRCP_2wkavg_1wk',
       'AWND_2wkavg_1wk', 'chlA_2wk', 'nitrate_2wk', 'phosphate_2wk',
       'silicate_2wk', 'nitrite_2wk', 'nitrate_phosphate_ratio_2wk',
       'N_star_2wk', 'Si_star_2wk', 'AvgTemp_1wk_2wk', 'AvgDens_1wk_2wk',
       'SST_2wk', 'WIND_SPD_2wk', 'U_WIND_2wk', 'PRCP_2wkavg_2wk',
       'AWND_2wkavg_2wk'],
      dtype='object')

In [23]:
### FOR DATA FROM ETHAN DEYLE:
HAB_data = pd.read_csv('Data/orig_block.csv', index_col=0) #LOAD DATA
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
# HAB_data = HAB_data.set_index('time')
target = 'chlA' #change target if needed

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data.columns

#dropping columns with 1wk or 2wk
cols_to_drop = [col for col in HAB_data.columns if 
    '1wk' in col or
    '_2wk' in col]
cols_to_drop.remove('AvgTemp_1wk')
cols_to_drop.remove('AvgDens_1wk')

cols_to_drop += ['serial_day']
HAB_data = HAB_data.drop(columns = cols_to_drop)
HAB_data

,N_star,SST,Si_star,U_WIND,WIND_SPD,chlA,nitrate,nitrate_phosphate_ratio,nitrite,phosphate,silicate
0,-2.200,19.118428,2.06,0.398250,3.219574,0.430,2.520,8.542373,0.020,0.295,2.355
1,-2.770,19.229762,2.30,0.701107,3.243543,0.920,2.110,6.918033,0.115,0.305,2.605
2,-4.005,19.083670,0.08,0.652186,2.679270,0.615,1.915,5.175676,0.020,0.370,0.450
3,-5.125,18.651255,1.17,0.735618,2.754969,0.655,0.155,0.469697,0.020,0.330,1.500
4,-4.920,18.196674,1.68,0.583428,2.934498,0.410,0.200,0.625000,0.030,0.320,2.000
...,...,...,...,...,...,...,...,...,...,...,...
1836,-3.540,14.007738,4.65,0.773389,4.063095,1.790,0.940,3.357143,0.120,0.280,4.930
1837,-2.840,13.920575,7.24,0.868929,4.294345,1.995,4.360,9.688889,0.270,0.450,7.690
1838,-3.470,13.516654,4.40,1.807142,4.558411,8.210,1.490,4.806452,0.110,0.310,4.710
1839,-5.840,13.329387,5.68,0.621545,4.609058,2.775,3.760,6.266667,0.230,0.600,6.280


In [6]:
#HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
HAB_data = HAB_data.set_index('time')
target = 'Avg_Chloro' #change target if needed

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data

KeyError: "None of ['time'] are in the columns"

In [10]:
HAB_data.columns

Index(['AWND_2wkavg', 'AWND_2wkavg_1wk', 'AWND_2wkavg_2wk', 'AvgDens_1wk',
       'AvgDens_1wk_1wk', 'AvgDens_1wk_2wk', 'AvgTemp_1wk', 'AvgTemp_1wk_1wk',
       'AvgTemp_1wk_2wk', 'N_star', 'N_star_1wk', 'N_star_2wk', 'PRCP_2wkavg',
       'PRCP_2wkavg_1wk', 'PRCP_2wkavg_2wk', 'SST', 'SST_1wk', 'SST_2wk',
       'Si_star', 'Si_star_1wk', 'Si_star_2wk', 'U_WIND', 'U_WIND_1wk',
       'U_WIND_2wk', 'WIND_SPD', 'WIND_SPD_1wk', 'WIND_SPD_2wk', 'chlA',
       'chlA_.1wk', 'chlA_1wk', 'chlA_2wk', 'nitrate', 'nitrate_1wk',
       'nitrate_2wk', 'nitrate_phosphate_ratio', 'nitrate_phosphate_ratio_1wk',
       'nitrate_phosphate_ratio_2wk', 'nitrite', 'nitrite_1wk', 'nitrite_2wk',
       'phosphate', 'phosphate_1wk', 'phosphate_2wk', 'serial_day', 'silicate',
       'silicate_1wk', 'silicate_2wk'],
      dtype='object')

In [7]:
print(HAB_data.isna().sum())

AVG_DENS_kgm3               198
AVG_SAL_PSU                 190
AVG_TEMP_C                  193
Akashiwo_sanguinea          768
Ammonium                    100
Avg_Chloro                    2
Avg_Phaeo                     2
BOT_DENS_kgm3               195
BOT_SAL_PSU                 187
BOT_TEMP_C                  192
CellCountDetection_Limit    768
Chl1                          3
Chl2                          3
Lingulodinium_polyedra      768
Nitrate                      99
Nitrite                      99
Phaeo1                        3
Phaeo2                        3
Phosphate                    99
SURF_DENS_kgm3              112
SURF_SAL_PSU                105
SURF_TEMP_C                 110
Silicate                     99
Temp                          0
Total_Cochlodinium_spp      768
Total_Diatoms               768
Total_Dinoflagellates       768
Total_Phytoplankton         768
Total_Prorocentrum_spp      768
Total_Tripos                906
WSPD                          0
index   

In [8]:
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
HAB_data = HAB_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(HAB_data)
HAB_data = Imputer.transform(HAB_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
HAB_data

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_99120/137097351.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_99120/137097351.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


,AVG_DENS_kgm3,AVG_SAL_PSU,AVG_TEMP_C,Akashiwo_sanguinea,Ammonium,Avg_Chloro,Avg_Phaeo,BOT_DENS_kgm3,BOT_SAL_PSU,BOT_TEMP_C,...,Temp,Total_Cochlodinium_spp,Total_Diatoms,Total_Dinoflagellates,Total_Phytoplankton,Total_Prorocentrum_spp,Total_Tripos,WSPD,index,level_0
time,,,,,,,,,,,,,,,,,,,,,
0,1024.129374,33.6950,18.10,0.0,0.530,1.82,0.60,1024.620655,33.62,15.9,...,19.8,0.0,69952.0,15684.0,85636.0,5168.0,0.0,4.691667,0.0,0.0
1,1024.094707,33.6700,18.20,44.5,0.795,2.08,0.53,1024.469221,33.60,16.5,...,19.7,0.0,118740.5,10693.5,129434.5,3163.0,0.0,3.954167,1.0,1.0
2,1024.289049,33.6400,17.35,89.0,1.060,2.71,1.18,1024.391719,33.62,16.9,...,17.3,0.0,167529.0,5703.0,173233.0,1158.0,0.0,2.787500,2.0,2.0
3,1023.591318,33.4850,9.90,89.0,0.705,1.32,0.52,1023.661703,33.53,19.6,...,19.5,133.5,117315.0,8109.0,125424.5,1113.5,0.0,2.825000,3.0,3.0
4,1023.563298,33.6600,20.25,89.0,0.350,0.99,0.55,1024.026040,33.58,18.3,...,21.7,267.0,67101.0,10515.0,77616.0,1069.0,0.0,3.412500,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,1024.940230,33.3700,13.45,0.0,0.410,3.38,1.00,1024.953741,33.36,13.4,...,13.8,0.0,95122.0,63206.0,158328.0,1252.0,26910.0,5.088889,1597.0,1597.0
1598,1024.789575,33.3600,14.15,0.0,0.410,6.75,0.88,1024.824150,33.35,14.0,...,14.2,0.0,191182.5,114209.0,305391.5,11890.5,51003.0,2.736806,1598.0,1598.0
1599,1024.900555,33.3725,13.65,0.0,0.410,9.71,1.49,1024.912716,33.36,13.6,...,13.2,0.0,287243.0,165212.0,452455.0,22529.0,75096.0,2.813194,1599.0,1599.0


In [9]:
print(HAB_data.isna().sum())

AVG_DENS_kgm3               0
AVG_SAL_PSU                 0
AVG_TEMP_C                  0
Akashiwo_sanguinea          0
Ammonium                    0
Avg_Chloro                  0
Avg_Phaeo                   0
BOT_DENS_kgm3               0
BOT_SAL_PSU                 0
BOT_TEMP_C                  0
CellCountDetection_Limit    0
Chl1                        0
Chl2                        0
Lingulodinium_polyedra      0
Nitrate                     0
Nitrite                     0
Phaeo1                      0
Phaeo2                      0
Phosphate                   0
SURF_DENS_kgm3              0
SURF_SAL_PSU                0
SURF_TEMP_C                 0
Silicate                    0
Temp                        0
Total_Cochlodinium_spp      0
Total_Diatoms               0
Total_Dinoflagellates       0
Total_Phytoplankton         0
Total_Prorocentrum_spp      0
Total_Tripos                0
WSPD                        0
index                       0
level_0                     0
dtype: int

In [10]:
#STANDARDIZE
#LEFT OFF HERE THERE IS AN ERROR
'''
scaler = MinMaxScaler()
columns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', 
           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', 
           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', 
           'Silicate', 'WSPD']
HAB_data[columns] = scaler.fit_transform(HAB_data[columns])
HAB_data
'''

"\nscaler = MinMaxScaler()\ncolumns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', \n           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', \n           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', \n           'Silicate', 'WSPD']\nHAB_data[columns] = scaler.fit_transform(HAB_data[columns])\nHAB_data\n"

In [11]:
#HAB_data.drop(columns=[]) # FOR MELISSA CARTER DATA
block = get_block(HAB_data, num_lags=50, tau=1)
block

,AVG_DENS_kgm3(t-0),AVG_SAL_PSU(t-0),AVG_TEMP_C(t-0),Akashiwo_sanguinea(t-0),Ammonium(t-0),Avg_Chloro(t-0),Avg_Phaeo(t-0),BOT_DENS_kgm3(t-0),BOT_SAL_PSU(t-0),BOT_TEMP_C(t-0),...,Temp(t-50),Total_Cochlodinium_spp(t-50),Total_Diatoms(t-50),Total_Dinoflagellates(t-50),Total_Phytoplankton(t-50),Total_Prorocentrum_spp(t-50),Total_Tripos(t-50),WSPD(t-50),index(t-50),level_0(t-50)
time,,,,,,,,,,,,,,,,,,,,,
0,1024.129374,33.6950,18.10,0.0,0.530,1.82,0.60,1024.620655,33.62,15.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1024.094707,33.6700,18.20,44.5,0.795,2.08,0.53,1024.469221,33.60,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1024.289049,33.6400,17.35,89.0,1.060,2.71,1.18,1024.391719,33.62,16.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1023.591318,33.4850,9.90,89.0,0.705,1.32,0.52,1023.661703,33.53,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1023.563298,33.6600,20.25,89.0,0.350,0.99,0.55,1024.026040,33.58,18.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,1024.940230,33.3700,13.45,0.0,0.410,3.38,1.00,1024.953741,33.36,13.4,...,17.5,9387.0,71341.0,149567.0,220908.0,64458.0,0.0,1.965278,1547.0,1547.0
1598,1024.789575,33.3600,14.15,0.0,0.410,6.75,0.88,1024.824150,33.35,14.0,...,16.7,3755.0,89490.0,120780.0,210270.0,28787.0,1252.0,5.076389,1548.0,1548.0
1599,1024.900555,33.3725,13.65,0.0,0.410,9.71,1.49,1024.912716,33.36,13.6,...,17.6,3755.0,92619.0,126099.0,218718.0,26909.5,626.0,3.731250,1549.0,1549.0


In [12]:
E_list = range(2,13)#usually 13
tau_list = [-1,-2,-3] #can try more taus
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
print(interactions)
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=2)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

[('AVG_DENS_kgm3', 'AVG_SAL_PSU'), ('AVG_DENS_kgm3', 'AVG_TEMP_C'), ('AVG_DENS_kgm3', 'Akashiwo_sanguinea'), ('AVG_DENS_kgm3', 'Ammonium'), ('AVG_DENS_kgm3', 'Avg_Chloro'), ('AVG_DENS_kgm3', 'Avg_Phaeo'), ('AVG_DENS_kgm3', 'BOT_DENS_kgm3'), ('AVG_DENS_kgm3', 'BOT_SAL_PSU'), ('AVG_DENS_kgm3', 'BOT_TEMP_C'), ('AVG_DENS_kgm3', 'CellCountDetection_Limit'), ('AVG_DENS_kgm3', 'Chl1'), ('AVG_DENS_kgm3', 'Chl2'), ('AVG_DENS_kgm3', 'Lingulodinium_polyedra'), ('AVG_DENS_kgm3', 'Nitrate'), ('AVG_DENS_kgm3', 'Nitrite'), ('AVG_DENS_kgm3', 'Phaeo1'), ('AVG_DENS_kgm3', 'Phaeo2'), ('AVG_DENS_kgm3', 'Phosphate'), ('AVG_DENS_kgm3', 'SURF_DENS_kgm3'), ('AVG_DENS_kgm3', 'SURF_SAL_PSU'), ('AVG_DENS_kgm3', 'SURF_TEMP_C'), ('AVG_DENS_kgm3', 'Silicate'), ('AVG_DENS_kgm3', 'Temp'), ('AVG_DENS_kgm3', 'Total_Cochlodinium_spp'), ('AVG_DENS_kgm3', 'Total_Diatoms'), ('AVG_DENS_kgm3', 'Total_Dinoflagellates'), ('AVG_DENS_kgm3', 'Total_Phytoplankton'), ('AVG_DENS_kgm3', 'Total_Prorocentrum_spp'), ('AVG_DENS_kgm3', 'T

('AVG_DENS_kgm3', 'Avg_Chloro')
('AVG_SAL_PSU', 'Avg_Chloro')
('AVG_TEMP_C', 'Avg_Chloro')
('Akashiwo_sanguinea', 'Avg_Chloro')
('Ammonium', 'Avg_Chloro')
('Avg_Chloro', 'AVG_DENS_kgm3')
('Avg_Chloro', 'AVG_SAL_PSU')
('Avg_Chloro', 'AVG_TEMP_C')
('Avg_Chloro', 'Akashiwo_sanguinea')
('Avg_Chloro', 'Ammonium')
('Avg_Chloro', 'Avg_Phaeo')
('Avg_Chloro', 'BOT_DENS_kgm3')
('Avg_Chloro', 'BOT_SAL_PSU')
('Avg_Chloro', 'BOT_TEMP_C')
('Avg_Chloro', 'CellCountDetection_Limit')
('Avg_Chloro', 'Chl1')
('Avg_Chloro', 'Chl2')
('Avg_Chloro', 'Lingulodinium_polyedra')
('Avg_Chloro', 'Nitrate')
('Avg_Chloro', 'Nitrite')


/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:119: RuntimeWarning: divide by zero encountered in divide
  distRowScale = self.theta / distRowMean
/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:120: RuntimeWarning: invalid value encountered in multiply
  W = exp( -distRowScale[:,None] * self.knn_distances )


 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Avg_Chloro', 'Nitrite') did not converge
('Avg_Chloro', 'Phaeo1')
('Avg_Chloro', 'Phaeo2')
('Avg_Chloro', 'Phosphate')
('Avg_Chloro', 'SURF_DENS_kgm3')
('Avg_Chloro', 'SURF_SAL_PSU')
('Avg_Chloro', 'SURF_TEMP_C')
('Avg_Chloro', 'Silicate')
('Avg_Chloro', 'Temp')
('Avg_Chloro', 'Total_Cochlodinium_spp')
('Avg_Chloro', 'Total_Diatoms')
('Avg_Chloro', 'Total_Dinoflagellates')
('Avg_Chloro', 'Total_Phytoplankton')
('Avg_Chloro', 'Total_Prorocentrum_spp')
('Avg_Chloro', 'Total_Tripos')
('Avg_Chloro', 'WSPD')
('Avg_Chloro', 'index')
('Avg_Chloro', 'level_0')
('Avg_Phaeo', 'Avg_Chloro')
('BOT_DENS_kgm3', 'Avg_Chloro')
('BOT_SAL_PSU', 'Avg_Chloro')
('BOT_TEMP_C', 'Avg_Chloro')
('CellCountDetection_Limit', 'Avg_Chloro')
('Chl1', 'Avg_Chloro')
('Chl2', 'Avg_Chloro')
('Lingulodinium_polyedra', 'Avg_Chloro')
('Nitrate', 'Avg_Chloro')
('Nitrite', 'Avg_Chloro')
('Phaeo1',

In [13]:
p_cutoff   = 0.2
rho_cutoff = 0.1

# filter to those rows where "lib (driven)" is your target
# and both p-value and CCM skill exceed your cutoffs
sig = results_df[
    (results_df.convergence_p_value < p_cutoff) &
    (results_df.ccm_value         > rho_cutoff) &
    (results_df['lib (driven)']   == target)
]

# extract the unique drivers
system_variables = sig['target (driver)'].unique().tolist()

print("Variables that significantly drive", target, ":")
print(system_variables)
sig

Variables that significantly drive Avg_Chloro :
['Akashiwo_sanguinea', 'Avg_Phaeo', 'CellCountDetection_Limit', 'Lingulodinium_polyedra', 'Nitrate', 'Phaeo1', 'Phaeo2', 'Silicate', 'Total_Diatoms', 'Total_Dinoflagellates', 'Total_Phytoplankton']


,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
3,Akashiwo_sanguinea,Avg_Chloro,2.0,-3.0,8.0,E tau theta rho 0 2.0 -1.0...,0.423364,0.016037,0.034048
37,Avg_Phaeo,Avg_Chloro,2.0,-3.0,0.0,E tau theta rho 0 2.0 -1.0...,0.877404,0.021690,0.891079
41,CellCountDetection_Limit,Avg_Chloro,5.0,-1.0,3.0,E tau theta rho 0 2.0 -1.0...,0.879920,0.000062,0.904477
44,Lingulodinium_polyedra,Avg_Chloro,2.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.969650,0.000024,0.963094
45,Nitrate,Avg_Chloro,7.0,-3.0,8.0,E tau theta rho 0 2.0 -1.0...,0.169863,0.040999,-0.031629
47,Phaeo1,Avg_Chloro,2.0,-3.0,0.0,E tau theta rho 0 2.0 -1.0...,0.850475,0.087147,0.868724
48,Phaeo2,Avg_Chloro,2.0,-3.0,0.0,E tau theta rho 0 2.0 -1.0...,0.903395,0.019913,0.916862
53,Silicate,Avg_Chloro,12.0,-3.0,2.0,E tau theta rho 0 2.0 -1.0...,0.374901,0.159598,0.195638
56,Total_Diatoms,Avg_Chloro,10.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.741714,0.002154,0.005475
57,Total_Dinoflagellates,Avg_Chloro,2.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.938090,0.033999,0.938624


In [16]:
results_df[results_df.convergence_p_value<0.3].sort_values(by='ccm_value', ascending=False)

,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
43,Chl2,Avg_Chloro,2.0,-2.0,2.0,E tau theta rho 0 2.0 -1.0...,0.996475,2.846823e-01,0.997727
15,Avg_Chloro,Chl1,2.0,-2.0,3.0,E tau theta rho 0 2.0 -1.0...,0.995706,2.612465e-01,0.997916
44,Lingulodinium_polyedra,Avg_Chloro,2.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.969650,2.355266e-05,0.963094
17,Avg_Chloro,Lingulodinium_polyedra,2.0,-1.0,8.0,E tau theta rho 0 2.0 -1.0...,0.964027,2.925542e-03,0.963094
57,Total_Dinoflagellates,Avg_Chloro,2.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.938090,3.399925e-02,0.938624
30,Avg_Chloro,Total_Dinoflagellates,2.0,-1.0,2.0,E tau theta rho 0 2.0 -1.0...,0.933684,9.506029e-04,0.938624
14,Avg_Chloro,CellCountDetection_Limit,5.0,-3.0,6.0,E tau theta rho 0 2.0 -1.0...,0.922070,4.214528e-02,0.904477
48,Phaeo2,Avg_Chloro,2.0,-3.0,0.0,E tau theta rho 0 2.0 -1.0...,0.903395,1.991320e-02,0.916862
31,Avg_Chloro,Total_Phytoplankton,2.0,-3.0,8.0,E tau theta rho 0 2.0 -1.0...,0.902716,5.850844e-05,0.850275
41,CellCountDetection_Limit,Avg_Chloro,5.0,-1.0,3.0,E tau theta rho 0 2.0 -1.0...,0.879920,6.190918e-05,0.904477
